In [1]:
import cv2
from skimage import io
from matplotlib import pyplot as plt
%matplotlib notebook
plt.ioff()
import tifffile, csv, numpy as np
from skimage.morphology import disk
from skimage.feature import canny
from skimage.filters import sobel

In [2]:
path_comm = "/media/ula/D/2p_fosgfp2020/result.tif"
out_name = "/media/ula/D/2p_fosgfp2020/m1s3_thre_cv.tif"

field_names_coords = ['X', 'Y', 'Z']
result_file = "/media/ula/D/2p_fosgfp2020/m1s3_thre_cv.csv"

rescale_coeff

img = io.imread(path_comm)
img.shape

(68, 512, 512)

In [8]:
with tifffile.TiffWriter(out_name) as tiff:   
    for im in img:
        ret, thresh = cv2.threshold(im, 0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        kernel = disk(1)
        opening = cv2.morphologyEx(255-thresh,cv2.MORPH_OPEN,kernel, iterations = 1)
        sure_bg = cv2.dilate(opening,kernel,iterations=1)
        dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,0)
        ret, sure_fg = cv2.threshold(dist_transform,0.3*dist_transform.max(),255,0)
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        ret, markers = cv2.connectedComponents(sure_fg)
        markers = markers+1
        markers[unknown==255] = 0
        rgb = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
        markers = cv2.watershed(rgb,markers)
        rgb[markers == -1] = [255,0,0]
        tiff.save(rgb)

200
134
151
156
177
128
129
146
184
140
161
176
214
198
153
121
166
138
153
166
141
167
184
147
166
168
194
202
172
171
190
221
195
186
124
141
164
180
175
163
169
173
182
181
172
181
139
132
125
91
86
77
61
46
28
27
18
13
11
7
8
8
10
11
9
3
6
4


In [7]:
help(cv2.watershed)

Help on built-in function watershed:

watershed(...)
    watershed(image, markers) -> markers
    .   @brief Performs a marker-based image segmentation using the watershed algorithm.
    .   
    .   The function implements one of the variants of watershed, non-parametric marker-based segmentation
    .   algorithm, described in @cite Meyer92 .
    .   
    .   Before passing the image to the function, you have to roughly outline the desired regions in the
    .   image markers with positive (\>0) indices. So, every region is represented as one or more connected
    .   components with the pixel values 1, 2, 3, and so on. Such markers can be retrieved from a binary
    .   mask using findContours and drawContours (see the watershed.cpp demo). The markers are "seeds" of
    .   the future image regions. All the other pixels in markers , whose relation to the outlined regions
    .   is not known and should be defined by the algorithm, should be set to 0's. In the function output,
    . 